In [1]:
# imports 
import os, time, json
import tmdbsimple as tmdb 
import pandas as pd
from tqdm.notebook import tqdm_notebook

In [2]:
def write_json(new_data, filename):
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [3]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)

    # save the .info. .releases dictionaries
    info = movie.info()
    releases = movie.releases()

    # Loop through countries in releases 
    for c in releases['countries']:
        # if the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            ## save a 'certification' key in info with the certification 
            info['certification'] = c['certification']
    return info

In [4]:
import json 
with open('D:/My Documents/GitHub/Stack-3/.secret/tmdb_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [5]:
import tmdbsimple as tmdb
tmdb.API_KEY = login['api-key']

In [7]:
basics = pd.read_csv('Data/basics.csv')
basics

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...,...
88541,10016149,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
88542,10016544,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
88543,10016684,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
88544,10016693,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [8]:
FOLDER = 'TMDB_Data/'
sorted(os.listdir(FOLDER))

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'tmdb_api_results_2000.json']

In [21]:
YEARS = [2001, 2002]
errors = []

In [34]:
# Define the JSON file to store results for the year
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEARS}.json'


# Check if the JSON file exists
file_exists = os.path.isfile(JSON_FILE)

# If it does not exist: create it
if file_exists == False:
    print(f"Creating {JSON_FILE} for API results for year={YEARS}.")
    
    # save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)

# If it exists, print a message
else:
    print(f'The file {JSON_FILE} already exists.')

Creating TMDB_Data/tmdb_api_results_[2001, 2002].json for API results for year=[2001, 2002].


In [33]:
df = basics.loc[(basics['startYear'] == 2001) | (basics['startYear'] == 2002)].copy()

# saving movie ids to seperate variable 
movie_ids = df['tconst']
len(movie_ids)

3315

In [36]:
previous_df = pd.read_json(JSON_FILE)
previous_df

,imdb_id
0,0


In [37]:
# filter out any ids that are alrady in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [38]:
# Loop through movie_ids_to_get with a tqdm progress bar 
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {'imdb_id'}"):

    # Attempt to retrieve then data for the movie id 
    try:
        temp = get_movie_with_rating(movie_id)
        write_json(temp, JSON_FILE)
        time.sleep(0.02)

    except Exception as e: 
        errors.append([movie_id, e])

Movies from imdb_id:   0%|          | 0/3315 [00:00<?, ?it/s]

In [40]:
import glob
# Use glob to get all filepaths that match the pattern (*=wildcard)
tmdb_files = sorted(glob.glob("TMDB_Data/final_tmdb_data*.csv.gz"))
tmdb_files

['TMDB_Data\\final_tmdb_data_2000.csv.gz']

In [44]:
# Save the final results to a csv.gz file
final_year_df = pd.read_json(JSON_FILE)

csv_fname = f"{FOLDER}final_tmdb_data_{YEARS}.csv.gz"
final_year_df.to_csv(csv_fname, compression="gzip", index=False)

In [45]:
import glob
# Use glob to get all filepaths that match the pattern (*=wildcard)
tmdb_files = sorted(glob.glob("TMDB_Data/final_tmdb_data*.csv.gz"))
tmdb_files

['TMDB_Data\\final_tmdb_data_2000.csv.gz',
 'TMDB_Data\\final_tmdb_data_[2001, 2002].csv.gz']

In [46]:
# Use read_csv in a list comprehension and combine with concat to load all files
df = pd.concat([pd.read_csv(f) for f in tmdb_files] )
df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,49666.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,24.0,PG
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,4.958,12.0,NaN
3,tt0116391,0.0,/krEZg9tb6blhc7sV6Us2ZGQ0gA.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,165.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,5.000,2.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843.0,cn,花樣年華,...,14204632.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.110,2476.0,PG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,tt8701182,0.0,NaN,NaN,0.0,"[{'id': 99, 'name': 'Documentary'}]",NaN,408492.0,en,still/here,...,0.0,60.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,still/here,0.0,9.000,1.0,NaN
2732,tt8795764,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",https://www.utahwolf.com/films/coming-soon-new...,871624.0,en,New Breed,...,0.0,57.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,New Breed,0.0,0.000,0.0,NR
2733,tt8825252,0.0,NaN,NaN,0.0,"[{'id': 14, 'name': 'Fantasy'}]",NaN,989195.0,en,Circle of Fire: The Dark Lord Kylnor,...,0.0,45.0,[],Released,NaN,Circle of Fire: The Dark Lord Kylnor,0.0,0.000,0.0,NaN
2734,tt9071078,0.0,/51274Q2vnmajonyiqq3YNQQ7q4I.jpg,NaN,127389.0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",https://movie.douban.com/subject/26420865/,1044371.0,zh,魔忍狂刀,...,0.0,100.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,Dragon Hero,Dragon Hero,0.0,0.000,0.0,NaN


In [49]:
# save combine years
csv_fname = f"{FOLDER}tmdb_results_combined.csv.gz" 
df.to_csv(csv_fname, compression = 'gzip', index = False)